<a href="https://colab.research.google.com/github/daonly/personal/blob/main/%5BConf%5D_physio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.2 MB/s eta 0:00:00


In [2]:
pip install bioread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=db9d48207f2dae31561979a45fbb62c773ca1832a3f24ae4340310cd8db34149
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [12]:
### preparation
import neurokit2 as nk
import bioread as br
import os
import pandas as pd
import numpy as np

### set path
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Res/Conformity/data')
datapath = r'/content/drive/MyDrive/Res/Conformity/data'
savepath = r'/content/drive/MyDrive/Res/Conformity/analyses'

Mounted at /content/drive


In [7]:
### import and set raw data: range 설정 필! 한 파일에 두번 돌리지 말 것.

tasklist = ['HRV', 'HCT', 'BlockI', 'BlockII']

for i in range(30, 31):

  txtpath = datapath+f'/sub{i}.txt'

  with open(txtpath, "r") as file:
      lines = file.readlines()
  with open(txtpath, "w") as file:
      file.writelines(lines[23:])

  data = pd.read_csv(txtpath, delimiter='\t', low_memory=False)
  data = data.iloc[1:]; data = data.iloc[:, :-1]

  signal = []
  for j in range(1, len(data)):
    if data['CH25'][j] == '0' and data['CH25'][j+1] == '5':
      signal.append(j+1)

  for j in range(4):
    slicedf = data.iloc[signal[2*j]:signal[2*j+1]]
    savefile = datapath+f'/sub{i}_{tasklist[j]}.csv'
    slicedf.to_csv(savefile, index=True)


In [9]:
### HRV: range 설정 필!

hrvdf = pd.DataFrame()

for i in range(23,24):

  ## SubID
  SubID = str(i)
  hrvdf.loc[0, 1] = SubID
  hrvdf.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf = pd.concat([hrvdf, hrvnonlin], axis=1, ignore_index=False)

for i in range(24,31):
  hrvdf1 = pd.DataFrame()

  ## SubID
  SubID = str(i)
  hrvdf1.loc[0, 1] = SubID
  hrvdf1.columns = ['SubID']

  ## Compute time-domain indices of HRV
  hrvfile = datapath+f'/sub{SubID}_HRV.csv'
  data = pd.read_csv(hrvfile)
  peaks, info = nk.ecg_peaks(data['CH1'], sampling_rate=2000)
  hrvtime = nk.hrv_time(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvtime], axis=1, ignore_index=False)

  ## Compute frequency-domain indices of HRV
  hrvfreq = nk.hrv_frequency(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvfreq], axis=1, ignore_index=False)

  ## Compute nonlinear indices of HRV
  hrvnonlin = nk.hrv_nonlinear(peaks, sampling_rate=2000, show=False)
  hrvdf1 = pd.concat([hrvdf1, hrvnonlin], axis=1, ignore_index=False)
  hrvdf = pd.concat([hrvdf, hrvdf1], ignore_index=True)

savefile = savepath+f'/HRVDataII.xlsx'
hrvdf.to_excel(savefile, index=False)

hrvdf

,SubID,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,23,684.764313,59.396280,31.462252,52.096658,29.559838,54.237521,NaN,NaN,38.768969,...,1.298215,8.053163,1.073146,1.429987,1.359295,2.002239,1.693461,1.844488,3.362789,0.706814
1,24,578.169082,22.656063,4.907846,21.256656,4.385996,21.466960,NaN,NaN,13.048745,...,1.017699,7.061521,0.819674,1.431930,1.333255,1.701903,1.308748,1.573904,2.229651,0.702234
2,25,711.799010,54.884969,34.608733,45.290938,30.397008,49.058789,NaN,NaN,28.414441,...,1.367485,7.969130,0.967384,1.486419,1.304329,1.899933,1.703825,1.712910,3.319958,0.622386
3,26,877.501222,71.915975,20.735145,69.066907,20.286023,70.086537,NaN,NaN,49.596040,...,1.678988,8.062981,1.135326,1.438020,1.318189,2.131361,1.758056,1.815213,3.205184,0.827292
4,27,576.032905,17.430850,7.089312,15.870377,1.041183,17.140106,NaN,NaN,7.833758,...,1.340347,6.926513,0.873506,1.470537,1.305899,2.193382,1.588284,1.658636,2.606307,0.596025
5,28,608.370558,43.701488,5.573140,43.586087,3.318918,43.681615,NaN,NaN,22.693723,...,0.704778,7.797863,0.691757,1.444620,1.303220,1.810873,1.460136,1.397390,2.410276,0.607569
6,29,754.525210,37.794491,13.273255,35.530196,10.660990,36.647651,NaN,NaN,28.360666,...,1.355372,7.612828,1.188311,1.555885,1.372976,2.181407,1.728107,1.831827,3.345104,0.840897
7,30,756.545263,50.491932,13.830071,48.670189,12.377504,49.582919,NaN,NaN,33.043508,...,1.442470,7.854239,1.106867,1.712173,1.403036,2.079808,1.693810,1.732452,3.132463,0.842380


In [13]:
### HCT: range 설정 필!

hctdf = pd.DataFrame()
hctdf.loc[1, 1] = '1'  #여기도 수정 필!
hctdf.columns = ['SubID']

for i in range(23,31):

  ## SubID
  SubID = str(i)
  hctdf.loc[i, 'SubID'] = SubID

  ## Extract basic features
  hctfile = datapath+f'/sub{SubID}_HCT.csv'
  data = pd.read_csv(hctfile)

  processed_df, info = nk.bio_process(ecg=data['CH1'], rsp=data['CH3'], sampling_rate=2000)

  ## Set epochs
  hctpath = datapath+f'/sub{i}_HCT.csv'
  data = pd.read_csv(hctpath, low_memory=False)

  startsignal = []; endsignal = []
  for j in range(len(data)-1):
    if data['CH21'][j] == 0 and data['CH21'][j+1] == 5:
      startsignal.append(j+1)
    elif data['CH21'][j] == 5 and data['CH21'][j+1] == 0:
      endsignal.append(j+1)

  ## count R-peaks in each epoch
  hctdf1 = pd.DataFrame()
  for j in range(6):
    slicedf = processed_df.iloc[startsignal[2*j]:endsignal[2*j]]
    rpeaks = slicedf['ECG_R_Peaks'].sum()
    hctdf1.loc[j+1, 1] = rpeaks

  ## calculate indices for interoceptive sensitivity
  rawcardf = pd.read_excel(datapath+f'/HCT/Cardiac-{i}.xls', header=1)
  rawcardf = rawcardf[['HC.RESP','HCCR.Slider1.Value']].iloc[2:8]
  rawcardf.index = range(1,7)
  hctdf1 = pd.concat([hctdf1, rawcardf], axis=1, ignore_index=False)
  hctdf1['stanIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])/hctdf1[1]
  hctdf1['alteIAcc'] = 1 - abs(hctdf1['HC.RESP']-hctdf1[1])*2/(hctdf1['HC.RESP']+hctdf1[1])
  hctdf1['alteCheck'] = hctdf1['HC.RESP'] - hctdf1[1]*2
  if (hctdf1['alteCheck'] > 0).any():
    print(f'Need alteIAcc analyses due to sub{i}')
  hctdf.loc[i, 'stanIAcc'] = hctdf1['stanIAcc'].mean()
  hctdf.loc[i, 'alteIAcc'] = hctdf1['alteIAcc'].mean()
  hctdf.loc[i, 'IS'] = hctdf1['HCCR.Slider1.Value'].mean()
  hctdf.loc[i, 'stanIAw'] = hctdf1['stanIAcc'].corr(hctdf1['HCCR.Slider1.Value'])
  hctdf.loc[i, 'alteIAw'] = hctdf1['alteIAcc'].corr(hctdf1['HCCR.Slider1.Value'])

hctdf = hctdf[['SubID', 'stanIAcc', 'alteIAcc', 'IS', 'stanIAw', 'alteIAw']]
savefile = savepath+f'/HCTData_PilotI&II.xlsx'
hctdf.to_excel(savefile, index=False)

hctdf

,SubID,stanIAcc,alteIAcc,IS,stanIAw,alteIAw
1,1,NaN,NaN,NaN,NaN,NaN
23,23,0.579579,0.464884,3.833333,0.425663,0.421552
24,24,0.528591,0.380076,6.666667,-0.527910,-0.536001
25,25,0.420151,0.169048,3.833333,-0.082645,-0.053445
26,26,0.853535,0.841149,7.500000,0.633710,0.624288
27,27,0.529096,0.377989,3.333333,0.130349,0.167222
28,28,0.668040,0.585533,5.666667,-0.278294,-0.253795
29,29,0.823625,0.802054,5.500000,0.193422,0.171120
30,30,0.673031,0.585558,5.333333,0.937809,0.926768


In [ ]:
hctdf1

,1,HC.RESP,HCCR.Slider1.Value,stanIAcc,alteIAcc,alteCheck
1,81.0,29.0,4.0,-0.793103,0.054545,-133.0
2,91.0,23.0,3.0,-1.956522,-0.192982,-159.0
3,75.0,37.0,7.0,-0.027027,0.321429,-113.0
4,66.0,34.0,9.0,0.058824,0.360000,-98.0
5,57.0,21.0,2.0,-0.714286,0.076923,-93.0
6,49.0,14.0,3.0,-1.500000,-0.111111,-84.0
